# GPU Check

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
  print(gpu.name, gpu.memoryTotal)

# Imports

In [ ]:
from search_eval.eval_OneShot import Eval_OS
from search_eval.optimizer.SingleImageDataset import SingleImageDataset
from search_eval.utils.common_utils import *
from search_space.search_space import SearchSpace

from nni import trace
import nni.retiarii.strategy as strategy
import nni.retiarii.serializer as serializer

from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader
from nni.retiarii.strategy import DARTS as DartsStrategy

import torch

torch.cuda.empty_cache()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# Strategy

In [ ]:
# Select the Search Strategy
strategy = DartsStrategy()
# strategy = strategy.DartsStrategy()
# strategy = strategy.ENAS()
# strategy = strategy.GumbelDARTS()
# strategy = strategy.RandomOneShot()

# oneshot

In [ ]:
total_iterations = 1200

resolution = 64
noise_type = 'gaussian'
noise_level = .09
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{45}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{45}.npy')

# Create the lightning module
module = Eval_OS(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=0.01, 
                buffer_size=100,
                patience=1000,
                weight_decay= 5e-7,

                show_every=200,
                report_every=25,
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)
val_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=val_loader)


# Create a Search Space
model_space = SearchSpace(depth=2)

# fast_dev_run=False

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=strategy)
experiment.run(config)

In [ ]:
# stop experiment and clear cache
experiment.stop()
torch.cuda.empty_cache()

In [ ]:

exported_arch = experiment.export_top_models()

exported_arch


In [ ]:
# Create the lightning module
from search_eval.eval_no_search_SGLD_ES import Eval_SGLD_ES
module = Eval_SGLD_ES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                learning_rate=0.01, 
                buffer_size=100,
                patience=1000,
                weight_decay= 5e-7,

                show_every=200,
                report_every=25,
                )

# test UNETSpace

In [ ]:
from search_eval.eval_generic import SGLDES
from search_eval.optimizer.SingleImageDataset import SingleImageDataset
from search_eval.utils.common_utils import *
from search_space.unet.unetspaceOS import UNetSpace

from nni import trace
import nni.retiarii.strategy as strategy
import nni.retiarii.serializer as serializer

from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader

import torch

torch.cuda.empty_cache()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

In [ ]:


total_iterations = 5000

resolution = 64
noise_type = 'gaussian'
noise_level = .09
img_id = np.random.randint(0, 50)

phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{img_id}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img_id}.npy')

learning_rate = 0.11
buffer_size = 1000
patience = 3000
weight_decay = 5e-7
show_every = 200
report_every = 25

# Create the lightning module
module = SGLDES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=learning_rate, # consider .01
                buffer_size=buffer_size,
                patience=patience,
                weight_decay= weight_decay,

                show_every=show_every,
                report_every=report_every,
                HPO=False,
                NAS=True,
                OneShot=True,
                SGLD_regularize=False,
                ES=True,
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)
val_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=val_loader)


# Select the Search Strategy
search_strategy = strategy.DARTS()
# search_strategy = strategy.ENAS()
# search_strategy = strategy.GumbelDARTS()
# search_strategy = strategy.RandomOneShot()


model_space = UNetSpace(
         C_in=1, 
         C_out=1, 
         depth=4, 
         nodes_per_layer=2, # accept only 1 or 2,
         ops_per_node=4,
         use_attention=True,

        )
# fast_dev_run=False

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=search_strategy)
experiment.run(config)

In [3]:
experiment.stop()

[2023-09-12 21:02:23] Stopping experiment, please wait...
[2023-09-12 21:02:23] Experiment stopped


In [ ]:

exported_arch = experiment.export_top_models(formatter='dict')

exported_arch[0]

In [ ]:
from search_space.unet.unetspaceOS import exportedUNet

exportNet = exportedUNet(exported_arch=exported_arch[0], depth=4, C_in=1, C_out=1)
exportNet.test()

In [ ]:
model_space = UNetSpace(
         C_in=1, 
         C_out=1, 
         depth=4, 
         nodes_per_layer=2, # accept only 1 or 2,
         ops_per_node=1,
         use_attention=False,

        )

model_space.test()